# Make movie of an experimental campaign

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import xarray as xr

import pynsitu as pin

crs = pin.maps.crs

import pynsitu.movies as movies

---
## generate sample data

In [14]:
start = "2018-01-01"
end = "2018-01-15"

lonc, latc = 30, 0
scale = 111e3

# drifters
def generate_drifter_data(drifter_id):
    """Create a drifter time series."""
    time = pd.date_range(start="2018-01-02", end="2018-01-15", freq="1H")
    v = 0.1  # m/s approx
    ux = v * np.random.randn(time.size)
    uy = v * np.random.randn(time.size)
    df = pd.DataFrame(dict(u=ux, v=uy, time=time))
    dt = (time[1] - time[0]) / pd.Timedelta("1s")
    df["lon"] = df["u"].cumsum() * dt / scale / np.cos(lonc * np.pi / 180)
    df["lat"] = df["v"].cumsum() * dt / scale
    # lon = v * np.cos(2 * np.pi * ((time - time[0]) / time_scale)) / scale
    # lat = v * np.sin(2 * np.pi * ((time - time[0]) / time_scale)) / scale
    # df = pd.DataFrame(dict(lon=lonc + lon, lat=latc + lat, time=time))
    df["id"] = drifter_id
    df = df.set_index("time")
    return df


dr = pd.concat([generate_drifter_data(i) for i in range(0, 5)])

# ships
def generate_ship_data():
    """Create a ship time series."""
    time = pd.date_range(start="2018-01-05", end="2018-01-10", freq="30T")
    v = 10  # m/s approx
    time_scale = pd.Timedelta("1D")
    lon = v * np.cos(2 * np.pi * ((time - time[0]) / time_scale)) / scale
    lat = v * np.sin(2 * np.pi * ((time - time[0]) / time_scale)) / scale
    df = pd.DataFrame(dict(lon=lonc + lon, lat=latc + lat, time=time))
    df["id"] = "myid"
    df = df.set_index("time")
    return df


ship = generate_ship_data()

# wind

---

## prepare for movie generation


### massage data

In [16]:
# map_gen = lambda : cp.map(extent=bounds, rivers=False, tile=("toner-lite", 12), land=False, bathy=False, coastline=False)

# or dynamic adjustment of extent
dl = 1
extent = (lonc - dl, lonc + dl, latc - dl, latc + df)  # fixed extent
# extent = None # dynamic adjustment of extent
map_gen = lambda extent: cp.map(
    extent=None,
    rivers=False,
    tile=("toner-lite", 12),
    land=False,
    bathy=False,
    coastline=False,
    figsize=(8, 8),
)
if extent is None:
    # dynamic adjustment of extent
    extent = dict(buffer=0.1, aspect_ratio=(2, 1), exclude=["wind"])


dr_keys = list(dr["id"].unique())
colorsd = {d: c for d, c in zip(dr_keys, pin.get_cmap_colors(len(dr_keys)))}
mv_dr = dict(
    dtype="drifter", data=dr, colors=colorsd, dt_trail="1H", head_style=dict(alpha=0.5)
)

mv_ship = dict(
    dtype="moving",
    data=ship,
    color="k",
    dt_trail="1H",
)

mv_wd = dict(
    dtype="vector_field",
    # data=wind,
    u="u10m",
    v="v10m",
    uref=10,
    di=3,
    scale=1e2,
    color="0.7",
)

### instantiate movie object

In [18]:
mv = movies.movie(
    start,
    end,
    "10T",
    map_gen,
    "figs",
    # extent=extent, #dynamic extent
    drifters=mv_dr,
    ship=mv_ship,
    # wind=mv_wd,
    # legend=True,
    legend="upper right",
)

In [19]:
mv.plot_snapshot(25);

TypeError: <lambda>() missing 1 required positional argument: 'extent'

### generate movie figures

In [ ]:
mv.make_figures(extents=new_extents)

### generate movie figures with geographical extent smoothing

In [32]:
extents = mv.dry_run()

# padd and smooth extents
new_extents = pad_smooth_extents(extents, 10)
# dev
# e = pd.DataFrame(padded_extents)
# ne = pd.DataFrame(new_extents)
# e[1].hvplot() * ne[1].hvplot()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 405/405 [00:18<00:00, 22.04it/s]


In [33]:
mv.make_figures(extents=new_extents)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 405/405 [10:12<00:00,  1.51s/it]


### transform figures into mp4

Requires ffmpeg to be installed, for example: `conda install ffmpeg`

In [34]:
movie.generate_mpg(mv.fig_dir, "movie")

movies should be ready at: /Users/aponte/Code/taos/insitu/taos2/taos2_drifters1.mp4
